In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup
import lxml
from bs4 import NavigableString
import re

In [2]:
# Must gather rating data for all the games without data in the ratings_df in the exploratory analysis notebook.
# Must gather positive, neutral, and negative ratings numbers for both users and critics as well as the
# metascore and userscore

In [3]:
no_rating_df = pd.read_csv("../Resources/games_to_get_ratings.csv")

In [4]:
!which chromedriver

/usr/local/bin/chromedriver


In [44]:
browser = Browser("chrome", executable_path="/usr/local/bin/chromedriver", headless=True)

In [6]:
games_df = pd.read_csv("../Resources/vgsales.csv.zip")
list_of_games = games_df["Name"].values.tolist()

In [ ]:
# Grabbing the Metascore

game_scores = {}
problem_games = []

for game in ["Super Smash Bros."]:
    browser.visit("https://www.metacritic.com/")
    # fill in my search term to the Metacritic Browser and us "\n" to press "enter" and initialize search
    browser.fill("search_term", game + "\n")
    html= browser.html
    my_soup = soup(html, "lxml")
    try:
        metascore = my_soup.find("a", class_="metascore_anchor").get_text()
        game_scores[game]= metascore
        print("SUCCESSFUL")
    except:
        problem_games.append(game)
        print("THIS IS A PROBLEM CHILD")

In [7]:
my_df = pd.DataFrame(game_scores, columns=["Name","Metascore"])

NameError: name 'game_scores' is not defined

In [8]:
## Navigating to the correct page via the search bar using splinter is too slow. I will rewrite the code to manually 
# write in the url using string interpolation. 
# I will need to interpolate the correct console name as well into the url. Metacritics url structure is:
# "https://www.metacritic.com/game/{console}/{game}"

In [9]:
# must change game names into the proper format to be interpolated into the URL.

game_conversions= {
    "PSV":"playstation-vita",
    "DC":"dreamcast",
    "WiiU":"wii-u",
    "GC":"gamecube",
    "N64":"nintendo-64",
    "XB":"xbox",
    "PSP":"psp",
    "PS4":"playstation-4",
    "PS": "playstation",
    "Wii":"wii",
    "PS3": "playstation-3",
    "PS2":"playstation-2",
    "GBA":"game-boy-advance",
    "DS":"ds",
    "XB":"xbox",
    "X360":"xbox-360"
}

def convert_to_name(platform_abb):
    if platform_abb in game_conversions:
        full_platform= game_conversions[platform_abb]
    else:
        full_platform="other"
    return full_platform
    

In [10]:
# This actually shows that the vast majority of our data belongs to the consoles listed on metacritic, these "legacy consoles"
# Get rid of those games which fall into the "other" category under full_platform_name
games_df["full_platform_name"]= games_df["Platform"].map(convert_to_name)
games_df = games_df.loc[games_df["full_platform_name"]!="other"]


In [ ]:
# Grabbing the Metascore

game_scores = {}
problem_games = []


# zip the two 
zipped_list =list(zip(games_df["full_platform_name"].tolist(),games_df["Name"].tolist()))

for platform, name in zipped_list:
    edited_game_title= name.lower().replace(":","-").replace(" ","-").replace("\W|\s","-").replace("--","-")
    browser.visit(f"https://www.metacritic.com/game/{platform}/{edited_game_title}")
    # fill in my search term to the Metacritic Browser and us "\n" to press "enter" and initialize search
    
    html= browser.html
    my_soup = soup(html, "lxml")
    try:
        metascore = my_soup.find("a", class_="metascore_anchor").get_text()
        game_scores[game]= metascore
        print("SUCCESSFUL")
    except:
        problem_games.append(game)
        print("THIS IS A PROBLEM CHILD")

In [ ]:
# This worked pretty well, but I will alter the code further see if I can increase my scraping success to fail ratio

In [ ]:
import re
mystring = "The Legend!of:Zelda; Breath of the wild"
x = re.compile("\W|\s")
mystring = re.sub(x,"-", mystring).lower().replace("--","-")
mystring

In [ ]:
# Grabbing the Metascore

game_scores = {}
problem_games = []
counter = 1

# zip the two 
zipped_list =list(zip(games_df["full_platform_name"].tolist(),games_df["Name"].tolist()))

for platform, name in zipped_list:
    x = re.compile("\W|\s")
    edited_game_title= re.sub(x,"-", name).lower().replace("--","-")
    # periods at the end of the url will show up as a dash. This conditional is to get rid of it.
    if edited_game_title[-1] == "-":
        edited_game_title = edited_game_title[:-1]
        print(edited_game_title)
    else:
        print(edited_game_title)
    browser.visit(f"https://www.metacritic.com/game/{platform}/{edited_game_title}")
    # fill in my search term to the Metacritic Browser and us "\n" to press "enter" and initialize search
    
    html= browser.html
    my_soup = soup(html, "lxml")
    try:
        metascore = my_soup.find("a", class_="metascore_anchor").get_text()
        game_scores[game]= metascore
        print("SUCCESSFUL")
        print(counter)
        counter+=1
    except:
        problem_games.append(game)
        print("THIS IS A PROBLEM CHILD")
        print(counter)
        counter+=1

In [45]:

problem_games = []
abberant_side_detail_games = []
game_info_list = []
counter = 1

# zip the two 
zipped_list =list(zip(games_df["full_platform_name"].tolist(),games_df["Name"].tolist()))

for platform, name in zipped_list:
    game_scores = {}
    x = re.compile("\W|\s")
    edited_game_title= re.sub(x,"-", name).lower().replace("--","-")
    # periods at the end of the url will show up as a dash. This conditional is to get rid of it.
    if edited_game_title[-1] == "-":
        edited_game_title = edited_game_title[:-1]
        print(f"\nGAME TITLE: {edited_game_title}")
    else:
        print(f"\nGAME TITLE: {edited_game_title}")
    browser.visit(f"https://www.metacritic.com/game/{platform}/{edited_game_title}")
    # fill in my search term to the Metacritic Browser and us "\n" to press "enter" and initialize search
    
    game_scores["Name"]=name
    
    html= browser.html
    my_soup = soup(html, "lxml")
    # Grabbing the Metascore
    try:
        metascore = my_soup.find("a", class_="metascore_anchor").get_text()
        game_scores["metascore"]= metascore
    except:
        problem_games.append(name)
        print('THIS IS A PROBLEM CHILD')
        continue
    # Grabbing the Userscore
    try:
        userscore_first = my_soup.find("div", class_="details side_details")
        user_score = userscore_first.select("div.metascore_w")[0].get_text()
        game_scores["user_score"]= user_score
    except:
        problem_games.append(name)
        print("THIS IS A PROBLEM CHILD")
        continue
    # Grabbing the release data
    try:
        product_data = my_soup.find("div",class_="product_data")
        release_data = product_data.find("li", class_="summary_detail release_data")
        release_date = release_data.find("span",class_="data").get_text()
        game_scores["release_date"]=release_date
    except:
        problem_games.append(name)
        print("THIS IS A PROBLEM CHILD")
        continue
    # Grabbing the positive, mixed, and negative critic review numbers
    try:
        critic_reviews = []
        positive_scores = my_soup.find("div",class_="distribution_wrap")
        x = positive_scores.select("ol.score_counts.hover_none span.count")
        for i in x:
            critic_reviews.append(i.get_text())
        game_scores["positive_critics"] = critic_reviews[0]
        game_scores["neutral_critics"]=critic_reviews[1]
        game_scores["negative_critics"]=critic_reviews[2]
    except:
        problem_games.append(name)
        print('THIS IS A PROBLEM CHILD')
        continue
    # Grabbing the positive, mixed, and negative user review numbers
    try:
        user_reviews = []
        user_scores = my_soup.find("div",class_="user_reviews_module")
        drilling_down = user_scores.find("div",class_="distribution_wrap")
        ordered_list_html = drilling_down.select("ol.score_counts.hover_none span.count")
        for i in ordered_list_html:
            user_reviews.append(i.get_text())
        game_scores["positive_users"] = user_reviews[0]
        game_scores["neutral_users"]=user_reviews[1]
        game_scores["negative_users"]=user_reviews[2]
    except:
        problem_games.append(name)
        print("THIS IS A PROBLEM CHILD")
        continue
    # Grabbing side details
    try:
        side_details = my_soup.find_all("div", class_="details side_details")[1]
        unordered_list = side_details.select("ul.summary_details li")
        developer = unordered_list[0].find("span", class_="data").get_text()
        game_scores["developer"]=developer
        number_players = unordered_list[2].find("span",class_="data").get_text()
        game_scores["number_players"]=number_players
        rating = unordered_list[4].find("span", class_="data").get_text()
        game_scores["rating"]=rating
        game_info_list.append(game_scores)
        print('SUCCESSFUL')
        print(f"This is the {counter}th game in the list.")
        counter+=1
    except:
        game_info_list.append(game_scores)
        print("SUCCESS but no side details")
        continue
        
            
        

        
        


GAME TITLE: wii-sports
SUCCESSFUL
This is the 1th game in the list.

GAME TITLE: mario-kart-wii
SUCCESSFUL
This is the 2th game in the list.

GAME TITLE: wii-sports-resort
SUCCESSFUL
This is the 3th game in the list.

GAME TITLE: new-super-mario-bros
SUCCESSFUL
This is the 4th game in the list.

GAME TITLE: wii-play
SUCCESSFUL
This is the 5th game in the list.

GAME TITLE: new-super-mario-bros-wii
SUCCESSFUL
This is the 6th game in the list.

GAME TITLE: nintendogs
THIS IS A PROBLEM CHILD

GAME TITLE: mario-kart-ds
SUCCESSFUL
This is the 7th game in the list.

GAME TITLE: wii-fit


KeyboardInterrupt: 

In [46]:
game_info_list

[{'Name': 'Wii Sports',
  'metascore': '\n76\n',
  'user_score': '8.0',
  'release_date': 'Nov 19, 2006',
  'positive_critics': '31',
  'neutral_critics': '20',
  'negative_critics': '0',
  'positive_users': '105',
  'neutral_users': '40',
  'negative_users': '3',
  'developer': '\n                    Nintendo                ',
  'number_players': 'No Online Multiplayer',
  'rating': 'E'},
 {'Name': 'Mario Kart Wii',
  'metascore': '\n82\n',
  'user_score': '8.4',
  'release_date': 'Apr 27, 2008',
  'positive_critics': '65',
  'neutral_critics': '7',
  'negative_critics': '1',
  'positive_users': '235',
  'neutral_users': '62',
  'negative_users': '22',
  'developer': '\n                    Nintendo                ',
  'number_players': 'Up to 12 ',
  'rating': 'E'},
 {'Name': 'Wii Sports Resort',
  'metascore': '\n80\n',
  'user_score': '8.1',
  'release_date': 'Jul 26, 2009',
  'positive_critics': '60',
  'neutral_critics': '13',
  'negative_critics': '0',
  'positive_users': '62',
 

In [ ]:
# get developer, genre, number_players, rating, release_date

In [15]:
game_info_list

[{'Name': 'Wii Sports',
  'metascore': '\n76\n',
  'user_score': '8.0',
  'positive_critics': '31',
  'neutral_critics': '20',
  'negative_critics': '0',
  'positive_users': '105',
  'neutral_users': '40',
  'negative_users': '3'},
 {'Name': 'Mario Kart Wii',
  'metascore': '\n82\n',
  'user_score': '8.4',
  'positive_critics': '65',
  'neutral_critics': '7',
  'negative_critics': '1',
  'positive_users': '235',
  'neutral_users': '62',
  'negative_users': '22'},
 {'Name': 'Wii Sports Resort',
  'metascore': '\n80\n',
  'user_score': '8.1',
  'positive_critics': '60',
  'neutral_critics': '13',
  'negative_critics': '0',
  'positive_users': '62',
  'neutral_users': '15',
  'negative_users': '4'},
 {'Name': 'New Super Mario Bros.',
  'metascore': '\n89\n',
  'user_score': '8.5',
  'positive_critics': '62',
  'neutral_critics': '3',
  'negative_critics': '0',
  'positive_users': '137',
  'neutral_users': '28',
  'negative_users': '2'},
 {'Name': 'Wii Play',
  'metascore': '\n58\n',
  'us

In [41]:

browser.visit("https://www.metacritic.com/game/switch/the-legend-of-zelda-breath-of-the-wild")
html = browser.html
my_soup = soup(html, "lxml")
side_data_list = []
side_details = my_soup.find_all("div", class_="details side_details")[1]
unordered_list = side_details.select("ul.summary_details li")
developer = unordered_list[0].find("span", class_="data").get_text()
game_scores["developer"]=developer
number_players = unordered_list[2].find("span",class_="data").get_text()
game_scores["number_players"]=number_players
rating = unordered_list[4].find("span", class_="data").get_text()
game_scores["rating"]=rating

game_scores

{'developer': '\n                    Nintendo                ',
 'number_players': 'No Online Multiplayer',
 'rating': 'E10+'}

In [ ]:
game_scores["developer"]=side_data_list[0]
game_scores["number_players"]=side_data_list[2]
game_scores["rating"]=side_data_list[4]

In [ ]:
for item in unordered_list:
    mylabel = item.select_one("span.label").get_text()
    side_data_list.append(myinfo)
side_data_list

In [ ]:
developer = unordered_list[0]
x = developer.select_one("span.data").get_text()
print(x)
.select_one("span.data").get_text()
    side_data_list.append(mydata)
side_data_list